In [ ]:
import spacy
print(f'spaCy: {spacy.__version__}')
from spacy import displacy
import random
from spacy.matcher import PhraseMatcher
# import plac
from pathlib import Path

spaCy: 3.5.2


Pre-training spaCy

In [ ]:
nlp = spacy.load('en_core_web_sm')
def show_ents(doc): 
  if doc.ents: 
    for ent in doc.ents: print(ent.text+' (' +str(ent.start_char) +', '+ str(ent.end_char) +') -> '+ent.label_+ ' | '+str(spacy.explain(ent.label_))) 
  else: print('No named entities found.')

doc1 = nlp("I want to explore the Goa Gajah Temple. When is the best time to visit?") 
show_ents(doc1)

the Goa Gajah Temple (18, 38) -> FAC | Buildings, airports, highways, bridges, etc.


Custom NER for travel spot, hotel, and food in bali

In [ ]:
entity_dict = {
    'ATTRACTION': ['Ubud Monkey Forest', 'Tegalalang Rice Terrace', 'Tanah Lot Temple', 'Mount Batur', 'Tirta Empul Temple', 'Kelingking Beach', 'Bali Safari and Marine Park', 'Sekumpul Waterfall', 'Jatiluwih Rice Terraces', 'Uluwatu Temple'],
    'HOTEL': ['Ubud Hotel', 'Seminyak Resort', 'Kuta Beach Hotel', 'Nusa Dua Resort', 'Jimbaran Villa'],
    'LOC': ['Ubud', 'Seminyak', 'Kuta', 'Nusa Dua', 'Jimbaran']
}

In [ ]:
nlp = spacy.blank("en")  # Initialize blank English model
ner = nlp.create_pipe("ner")
nlp.add_pipe("ner")

In [ ]:
for label, values in entity_dict.items():
    for value in values:
        ner.add_label(label)

In [ ]:
# Convert TRAIN_DATA into spaCy format
train_examples = []
for text, annotations in TRAIN_DATA:
    entities = annotations["entities"]
    train_examples.append((text, {"entities": entities}))


In [ ]:
from spacy.util import minibatch, compounding
random.seed(42)  # For reproducibility
nlp.begin_training()

# Set the number of training iterations and batch sizes
n_iter = 10
batch_size = 8

from spacy.training.example import Example
# Training loop
for itn in range(n_iter):
    random.shuffle(train_examples)
    losses = {}
    batches = minibatch(train_examples, size=batch_size)
    for batch in batches:
        examples = []
        for text, annotations in batch:
            example = Example.from_dict(nlp.make_doc(text), annotations)
            examples.append(example)
        nlp.update(examples, losses=losses, drop=0.5)
    print("Iteration:", itn+1, "Loss:", losses)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "What are some popular scuba diving spots in Bali?" with entities "[(29, 43, 'ATTRACTION')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Can you recommend any budget-friendly hotels in Nu..." with entities "[(33, 45, 'HOTEL')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Are there any famous beaches in Jimbaran?" wit

Iteration: 1 Loss: {'ner': 382.29433422330163}
Iteration: 2 Loss: {'ner': 27.718862646075948}
Iteration: 3 Loss: {'ner': 24.694102102978306}
Iteration: 4 Loss: {'ner': 49.19866522133906}
Iteration: 5 Loss: {'ner': 23.93502083113348}
Iteration: 6 Loss: {'ner': 20.547945901929214}
Iteration: 7 Loss: {'ner': 18.372320893058134}
Iteration: 8 Loss: {'ner': 18.795182025443875}
Iteration: 9 Loss: {'ner': 12.032689501397822}
Iteration: 10 Loss: {'ner': 14.89108903174639}


In [ ]:
nlp.to_disk("custom_ner_model")

In [ ]:
nlp1 = spacy.load('custom_ner_model')
def show_ents(doc): 
  if doc.ents: 
    for ent in doc.ents: print(ent.text+' (' +str(ent.start_char) +', '+ str(ent.end_char) +') -> '+ent.label_+ ' | '+str(spacy.explain(ent.label_))) 
  else: print('No named entities found.')

doc1 = nlp1("I want to explore Bali. When is the best time to visit?") 
show_ents(doc1)

No named entities found.
